# Calculate the similarity of 204060 mzML

In [3]:
import pandas as pd
import numpy as np
import os
import mzbatch
from sklearn.metrics.pairwise import cosine_similarity
import importlib as imp
imp.reload(mzbatch)
import subprocess
import warnings
warnings.filterwarnings("ignore") 

In [4]:
root_path = f'../../Fulaoshi/230821_mzML/'
analysis_batch = "BloodYin240307"
rawfile_batch = "QCidentify_1"
ev13_list = [1,10,15,20,25,30,35,40,50,60,70,80,100]


In [2]:
"""load reference"""
ref_all_df = pd.read_csv("../../Fulaoshi/230821_mzML/ref_all_增加evlist和nums.csv",index_col=0)
ref_all_df['batch_sampleid'] = ref_all_df['batch']+'/'+ref_all_df['sampleid']
ref_all_df['bscp']=ref_all_df['batch']+'+'+ref_all_df['sampleid'].astype(str)+'+'+ref_all_df['CAS No.']+'+'+ref_all_df['pos_neg']
ref_all_df["filepath"] = "../../Fulaoshi/230821_mzML_res/rerun240410/"+ref_all_df["batch"]+"/NCE_compound_filter_1/"+ref_all_df["sampleid"].astype(str)+"/"+ref_all_df["CAS No."]+"_NCE204060_"+ref_all_df["pos_neg"]
print(ref_all_df.shape)
ref_all_df.head(1)

### Establishing Basic Data

In [ ]:
# Establishing four mz_list
Amide_pos_mzlist = ref_all_df[(ref_all_df["sepuzhu"] == "Amide") & (ref_all_df["pos_neg"] == "pos")]["m/z"].tolist()
Amide_pos_mzlist.sort()
Amide_neg_mzlist = ref_all_df[(ref_all_df["sepuzhu"] == "Amide") & (ref_all_df["pos_neg"] == "neg")]["m/z"].tolist()
Amide_neg_mzlist.sort()
C18_pos_mzlist = ref_all_df[(ref_all_df["sepuzhu"] == "C18") & (ref_all_df["pos_neg"] == "pos")]["m/z"].tolist()
C18_pos_mzlist.sort()
C18_neg_mzlist = ref_all_df[(ref_all_df["sepuzhu"] == "C18") & (ref_all_df["pos_neg"] == "neg")]["m/z"].tolist()
C18_neg_mzlist.sort()

# Establishing upper and lower limits for the four mz_lists
Ap_mzrange_list = [[i, mzbatch.get_mz_range(i, diff=15)] for i in Amide_pos_mzlist]
An_mzrange_list = [[i, mzbatch.get_mz_range(i, diff=15)] for i in Amide_neg_mzlist]
Cp_mzrange_list = [[i, mzbatch.get_mz_range(i, diff=15)] for i in C18_pos_mzlist]
Cn_mzrange_list = [[i, mzbatch.get_mz_range(i, diff=15)] for i in C18_neg_mzlist]

mzrange_dict = {"Amide_pos": Ap_mzrange_list, "Amide_neg": An_mzrange_list, 
                "C18_pos": Cp_mzrange_list, "C18_neg": Cn_mzrange_list}


In [7]:
import itertools
l1_list = ["Amide","C18"][0:]
l2_list = ["pos","neg"][0:]
folder_combina_list = list(itertools.product(l1_list, l2_list))
len(folder_combina_list)

4

In [8]:
"""Build the Area_cols_dict"""
Area_cols_dict={}
for temp_sepuzhu,temp_pos_neg in [['Amide', 'pos'],['Amide', 'neg'],['C18', 'pos'],['C18', 'neg']]:
    MS_DIAL_list = os.listdir(f"../../Fulaoshi/230821_mzML/{analysis_batch}_{temp_sepuzhu}_{temp_pos_neg}/MS-DIAL")
    Area13_filename_list = [i for i in MS_DIAL_list if i.startswith("Area")]
    Area13_df = pd.read_csv(f"../../Fulaoshi/230821_mzML/{analysis_batch}_{temp_sepuzhu}_{temp_pos_neg}/MS-DIAL/{Area13_filename_list[0]}", sep="\t",index_col=0)
    if analysis_batch=="BloodNaoLiu240807":
        Area_cols_list = [i for i in Area13_df.columns if ((i.endswith(f"-{temp_pos_neg}")&(i.count('-'))))]
    else:
        Area_cols_list = [i for i in Area13_df.columns if ((i.strip().endswith(f"-{temp_pos_neg}")&(i.count('-')==2)&(i.count('_')==0)))]
    Area_cols_dict[f"{temp_sepuzhu}_{temp_pos_neg}"] = Area_cols_list

### Calculate

In [ ]:
"""First filter the mz of 4 204060 mzML with the 4000 standard library"""
for folder_sepuzhu, folder_pos_neg in folder_combina_list[0:]:
    mzrange_list = mzrange_dict[f"{folder_sepuzhu}_{folder_pos_neg}"]
    analysis_path = f"{analysis_batch}_{folder_sepuzhu}_{folder_pos_neg}"
    
    mzML_path = f"{root_path}/{analysis_path}/mzML/{rawfile_batch}-NCE204060-{folder_pos_neg}.mzML"
    pkl_path = f"{root_path}/{analysis_path}/pkl/{rawfile_batch}-NCE204060-{folder_pos_neg}.pkl"
    pkl_filter_folderpath = f"{root_path}/{analysis_path}/pkl_filter/"
    pkl_filter_path = f"{root_path}/{analysis_path}/pkl_filter/{rawfile_batch}-NCE204060-{folder_pos_neg}.pkl"
    
    raw_list = mzbatch.get_TIC_from_File(mzML_path, pkl_path)
    raw_df = pd.DataFrame(raw_list)
    raw_ms2_df = raw_df[raw_df["MSlevel"] == 2]
    
    valid_index_list = []  # Result indices
    for theo_mz, (mz_low, mz_up) in mzrange_list:
        temp_df = raw_ms2_df[(raw_ms2_df['pecursor'] >= mz_low) & (raw_ms2_df['pecursor'] <= mz_up)]
        if temp_df.shape[0] > 0:
            valid_index_list.extend(temp_df.index.tolist())
        # break
    valid_index_list = list(set(valid_index_list))
    valid_index_list.sort()
    
    if not os.path.exists(pkl_filter_folderpath):
        os.makedirs(pkl_filter_folderpath)
        while os.path.exists(pkl_filter_folderpath):
            break
    raw_ms2_df.loc[valid_index_list, :].to_pickle(pkl_filter_path)

    print("raw size:", raw_ms2_df.loc[valid_index_list, :].shape)


raw大小： (4691, 9)
raw大小： (7073, 9)
raw大小： (8249, 9)
raw大小： (4681, 9)


In [ ]:
for sepuzhu, pos_neg in folder_combina_list[0:]: 
    analysis_path = f"{analysis_batch}_{sepuzhu}_{pos_neg}"
    pkl_filter_folderpath = f"{root_path}/{analysis_path}/pkl_filter/"

    qc_filename = f'../../Fulaoshi/230821_mzML/BloodYin240307_{sepuzhu}_{pos_neg}//MS-DIAL/QCidentify_1-NCE204060-{pos_neg}.txt'
    mzrange_list = mzrange_dict[f"{sepuzhu}_{pos_neg}"]
    
    QC246_df = pd.read_csv(qc_filename, index_col=0, sep="\t")
    QC246_df = QC246_df.loc[:, ["RT left(min)", "RT right (min)", "Precursor m/z", "Area"]]
    QC246_df["RT left(s)"] = QC246_df["RT left(min)"] * 60
    QC246_df["RT right(s)"] = QC246_df["RT right (min)"] * 60
    
    # Filter QC246 based on mz
    QC_valid_index_list = []  # Result indices
    theo_mz_list = []
    for theo_mz, (mz_low, mz_up) in mzrange_list[0:]:
        temp_QC246_df = QC246_df[(QC246_df['Precursor m/z'] >= mz_low) & (QC246_df['Precursor m/z'] <= mz_up)]
        if temp_QC246_df.shape[0] > 0:
            QC_valid_index_list.extend(temp_QC246_df.index.tolist())
            theo_mz_list.extend([theo_mz] * temp_QC246_df.shape[0])
    
    QC246_filter_df = QC246_df.loc[QC_valid_index_list, :].copy(deep=True)
    QC246_filter_df["theo_mz"] = theo_mz_list
    QC246_filter_df.drop_duplicates(subset=['Precursor m/z', 'RT left(s)', 'RT right(s)', 'theo_mz'], inplace=True)
    QC246_filter_df.sort_values(by="theo_mz", inplace=True)
    QC246_filter_df.reset_index(inplace=True, drop=True)

    # Output to file
    if not os.path.exists(pkl_filter_folderpath):
        os.makedirs(pkl_filter_folderpath)
        while os.path.exists(pkl_filter_folderpath):
            break
    QC246_filter_df.to_pickle(f"{pkl_filter_folderpath}/QC246_filter_df.pkl")


In [ ]:
"""Based on the combination of 13 QCS, the similarity is calculated for 204060 where mz is within 15ppm and rt is within the range"""
res_df_dict = {}
for sepuzhu, pos_neg in folder_combina_list[0:]:
    analysis_path = f"{analysis_batch}_{sepuzhu}_{pos_neg}"
    pkl_filter_folderpath = f"{root_path}/{analysis_path}/pkl_filter/"
    pkl_filter_path = f"{root_path}/{analysis_path}/pkl_filter/{rawfile_batch}-NCE204060-{pos_neg}.pkl"
    pkl_path = f"{root_path}/{analysis_path}/pkl/{rawfile_batch}-NCE204060-{pos_neg}.pkl"
    
    # Get MS1 data
    raw_df = pd.DataFrame(pd.read_pickle(pkl_path))
    raw_ms1_df = raw_df[raw_df["MSlevel"] == 1]
    
    # Read QC35 and raw_filter_ms2_df
    QC35_filter_df = pd.read_pickle(f"{pkl_filter_folderpath}/con_QC_filter_df.pkl")
    raw_filter_ms2_df = pd.read_pickle(pkl_filter_path)
    
    # Construct DataFrame to store results
    res_cols_list = ["theo_mz", "ref_ev", "ref_mz", "ref_rt_left", "ref_rt_right", "Area", "ms2_ev", "ms2_index", "ms2_mz", "ms2_rt", "ms1_intensity"]
    
    res_df = pd.DataFrame(columns=res_cols_list)
    ref_ev = 35
    n = 0
    for QC35_filter_index in QC35_filter_df.index[0:]:  # QC35 is all 1
        theo_mz, ref_mz, ref_rt_left, ref_rt_right, Area = QC35_filter_df.loc[QC35_filter_index, ["theo_mz", 'Precursor m/z', "RT left(s)", "RT right(s)", "Area"]]
        ref_mz_low, ref_mz_up = mzbatch.get_mz_range(ref_mz, diff=15)

        ms2_ev = "204060"
        ms2_temp_df = raw_filter_ms2_df[((raw_filter_ms2_df['pecursor'] >= ref_mz_low) & (raw_filter_ms2_df['pecursor'] <= ref_mz_up)) 
                                          & ((raw_filter_ms2_df['RT'] >= ref_rt_left) & (raw_filter_ms2_df['RT'] <= ref_rt_right))].copy(deep=True)
        if ms2_temp_df.shape[0] > 0:
            for ms2_index in ms2_temp_df.index:
                ms2_mz, ms2_rt, ms2_peaks = ms2_temp_df.loc[ms2_index, ["pecursor", "RT", "peaks"]]
                ms2_mz_low, ms2_mz_up = mzbatch.get_mz_range(ms2_mz)
                # Calculate MS1 intensity
                ms1_df = pd.DataFrame(raw_ms1_df[raw_ms1_df["RT"] < ms2_rt].iloc[-1, -2], columns=["Mass", "Intensity"])
                ms1_intensity_filter_df = ms1_df[(ms1_df['Mass'] >= ms2_mz_low) & (ms1_df['Mass'] <= ms2_mz_up)]
                if ms1_intensity_filter_df.shape[0] > 0:
                    ms1_intensity = ms1_intensity_filter_df["Intensity"].max()
                else:
                    ms1_intensity = -1
                nce_compound_204060_folder = os.path.dirname(f'../../Fulaoshi/230821_mzML_res/{analysis_path}/NCE_compound_204060/{ms2_index}.csv')
                if not os.path.exists(nce_compound_204060_folder):
                    os.makedirs(nce_compound_204060_folder)
                    while os.path.exists(nce_compound_204060_folder):
                        break
                pd.DataFrame(ms2_temp_df.loc[ms2_index, "peaks"], columns=["Mass", "Intensity"]).to_csv(f'../../Fulaoshi/230821_mzML_res/{analysis_path}/NCE_compound_204060/{ms2_index}.csv')  # Output MS2 data to file
                res_df.loc[n] = [theo_mz, ref_ev, ref_mz, ref_rt_left, ref_rt_right, Area, ms2_ev, ms2_index, ms2_mz, ms2_rt, ms1_intensity]
                n += 1
            # break
    # Discard -1, only keep the MS1 intensity maximum spectrum
    res_nofu1_df = res_df[res_df["ms1_intensity"] != -1]
    max_ms1_indexes = res_nofu1_df.groupby(by=['theo_mz', 'ref_rt_left', 'ref_rt_right', 'ms2_ev'])['ms1_intensity'].idxmax()
    res_maxms1_df = res_nofu1_df.loc[max_ms1_indexes, :]
    
    res_df_dict[f"{sepuzhu}+{pos_neg}"] = res_maxms1_df


In [29]:
def fangda(x):
    if x=='-1' or x=='[[]]' or x==-1:
        return [[-1]*Area_lens]
    else:
        return x
Area_lens = len(Area_cols_list)

In [ ]:
"""res_df adds Area13 columns"""
for sepuzhu, pos_neg in folder_combina_list[0:]:
    res_groupby_df = res_df_dict[f"{sepuzhu}+{pos_neg}"]
    MS_DIAL_list = os.listdir(f"../../Fulaoshi/230821_mzML/{analysis_batch}_{sepuzhu}_{pos_neg}/MS-DIAL")
    Area13_filename_list = [i for i in MS_DIAL_list if i.startswith("Area")]
    Area13_df = pd.read_csv(f"../../Fulaoshi/230821_mzML/{analysis_batch}_{sepuzhu}_{pos_neg}/MS-DIAL/{Area13_filename_list[0]}", sep="\t",index_col=0)
    Area13_df["Average Rt(s)"] = Area13_df["Average Rt(min)"]*60
    Area_cols_list = Area_cols_dict[f"{sepuzhu}_{pos_neg}"]
    Area13_3list = []
    for res_groupby_index in res_groupby_df.index[0:]:
        res_groupby_mz, res_groupby_rt_low, res_groupby_rt_up = res_groupby_df.loc[res_groupby_index,['ref_mz', 'ref_rt_left', 'ref_rt_right']] #240915修改为theo_mz
        res_groupby_mz_low, res_groupby_mz_up = mzbatch.get_mz_range(res_groupby_mz)

        temp_Area13_df = Area13_df[((Area13_df['Average Mz'] >= res_groupby_mz_low) & (Area13_df['Average Mz'] <= res_groupby_mz_up))
                  & ((Area13_df['Average Rt(s)'] >= res_groupby_rt_low) & (Area13_df['Average Rt(s)'] <= res_groupby_rt_up))]
        if temp_Area13_df.shape[0]==1:
            Area13_3list.append(temp_Area13_df.loc[:,Area_cols_list].values.tolist())
        elif temp_Area13_df.shape[0]>1:
            Area13_3list.append(temp_Area13_df.loc[:,Area_cols_list].values.tolist())
        else:
            Area13_3list.append(-1)
    res_groupby_df["Area13_2list"] = Area13_3list
    # handle
    res_groupby_df["Area13_2list"] = res_groupby_df["Area13_2list"].apply(lambda x:fangda(x))
    res_groupby_df["Area13_2list"] = res_groupby_df["Area13_2list"].apply(lambda x:eval(str(x)))
    res_groupby_df = res_groupby_df.explode("Area13_2list") 
    res_groupby_df.reset_index(drop=True,inplace=True)
    
    res_df_dict[f"{sepuzhu}+{pos_neg}"] = res_groupby_df

In [ ]:
import pickle
with open(f"../../Fulaoshi/230821_mzML/res_df_dict.pkl", 'wb') as f:
    pickle.dump(res_df_dict, f)

print("The dictionary has been saved to ../../Fulaoshi/230821_mzML/res_df_dict.pkl")


字典已保存到 ../../Fulaoshi/230821_mzML/res_df_dict.pkl


In [ ]:
# Load the dictionary from the .pkl file
with open('../../Fulaoshi/230821_mzML/res_df_dict.pkl', 'rb') as f:
    res_df_dict = pickle.load(f)

print("The dictionary loaded from output.pkl is:", res_df_dict)


In [33]:
import itertools
l1_list = ["Amide","C18"][0:]
l2_list = ["pos","neg"][0:]
l3_list = ["cosine","jaccard","entropy"][0:]
combine3_list = list(itertools.product(l1_list, l2_list, l3_list))
len(combine3_list),combine3_list[0:5]

(12,
 [('Amide', 'pos', 'cosine'),
  ('Amide', 'pos', 'jaccard'),
  ('Amide', 'pos', 'entropy'),
  ('Amide', 'neg', 'cosine'),
  ('Amide', 'neg', 'jaccard')])

In [37]:
import numpy as np

def cal_fragment(fragment_mzrange_list, b2_mzlist):
    # Convert lists to NumPy arrays
    b2_mzarray = np.array(b2_mzlist)
    fragment_mzrange_array = np.array(fragment_mzrange_list) * 1e6
    # Separate the lower and upper bounds of the ranges
    ilow = fragment_mzrange_array[:, 0]
    iup = fragment_mzrange_array[:, 1]
    # Use boolean indexing to find elements that meet the conditions
    mask = (b2_mzarray[:, np.newaxis] >= ilow) & (b2_mzarray[:, np.newaxis] <= iup)
    # Find all m/z values that satisfy the conditions
    matched_indices = np.where(mask.any(axis=1))[0]
    fragment_mzlist = b2_mzarray[matched_indices]
    return fragment_mzlist

In [45]:
# Take ref_all as the standard diagram b1, and find the corresponding b2 from ref_df
for sepuzhu, pos_neg, sim_method in combine3_list[0:]:
    n = 0
    Area_cols_list = Area_cols_dict[f"{sepuzhu}_{pos_neg}"]
    hengxiang204060_df = pd.DataFrame(columns=["sepuzhu", "pos_neg", "bscp1", "b1_name", "b1_adduct", "b2_index", "b2_mz", "b2_rt", "pos_sim"] + Area_cols_list)
    analysis_path = f"{analysis_batch}_{sepuzhu}_{pos_neg}"
    pattern_ref_all_df = ref_all_df[(ref_all_df["sepuzhu"] == sepuzhu) & (ref_all_df["pos_neg"] == pos_neg)]
    res_df = res_df_dict[f"{sepuzhu}+{pos_neg}"]
    for i in res_df.index[0:]:
        ref_mz, ref_rt_left, ref_rt_right, ms2_ev, ms2_index, ms2_mz, ms2_rt, Area13_2list = res_df.loc[i, ["ref_mz", "ref_rt_left", "ref_rt_right", "ms2_ev", "ms2_index", "ms2_mz", "ms2_rt", "Area13_2list"]]
        # Area13_2list = eval(Area13_2list)
        ms2_mz_low, ms2_mz_up = mzbatch.get_mz_range(ms2_mz, diff=15)
        temp_pattern_ref_all_df = pattern_ref_all_df[(pattern_ref_all_df['m/z'] >= ms2_mz_low) & (pattern_ref_all_df['m/z'] <= ms2_mz_up)]
        # & (pattern_ref_all_df['RT time'] >= ref_rt_left) & (pattern_ref_all_df['RT time'] <= ref_rt_right)]
        if temp_pattern_ref_all_df.shape[0] > 0:
            for j in temp_pattern_ref_all_df.index[0:]:
                bscp1, b1_name, b1_adduct, b1_mz, temp_filepath = temp_pattern_ref_all_df.loc[j, ["bscp", "Product Name", "Adduct", "m/z", "filepath"]]
                batch1, sampleid1, cas1, pos_neg1 = bscp1.split("+")
                if ((not os.path.exists(temp_filepath)) or (os.listdir(temp_filepath) == [])):
                    continue
                temp_filename = os.listdir(temp_filepath)[0]
                b1_df = mzbatch.read_data(f"{temp_filepath}/{temp_filename}", b1_mz, rate=100)
                # b2_df = mzbatch.read_data(f"../../Fulaoshi/230821_mzML_res/{analysis_path}/NCE_compound_204060/{ms2_index}.csv", ms2_mz, rate=100) #240915 modification, add b2 fragment filtering
                
                # Filter b2 fragments: first find b1 fragments
                filter_fragment_path = f"../../Fulaoshi/230821_mzML_res/rerun240410/{batch1}/NCE_filter_fragment/{bscp1}.csv"
                if not os.path.exists(filter_fragment_path):
                    # print(f"{bscp1} has no fragments")
                    continue
                fragment_list = eval(pd.read_csv(filter_fragment_path, index_col=0)["filter_fragment"].tolist()[0])
                fragment_mzrange_list = mzbatch.get_mz_range_matrix(fragment_list, diff=15)
                # print(fragment_mzrange_list)
                
                # Read data, first read b2, filter b2 fragments and skip to the next loop if none are found
                # b2_df = pd.read_csv(f"../../Fulaoshi/230821_mzML_res/{analysis_path}/NCE_compound_204060/{ms2_index}.csv", index_col=0)
                b2_df = mzbatch.read_data(f"../../Fulaoshi/230821_mzML_res/{analysis_path}/NCE_compound_204060/{ms2_index}.csv", ms2_mz, rate=1000000) #240915 modification, add b2 fragment filtering
                # Filter b2 fragments
                b2_mzlist = b2_df.index.tolist()
                # print(len(b2_mzlist))
                fragment_res_list = cal_fragment(fragment_mzrange_list, b2_mzlist)
                # Skip similarity calculation with b1 if no fragments are found
                if len(fragment_res_list) == 0:
                    # print(f"Unknown spectrum {ms2_index} has no matching fragments, {len(fragment_res_list)}")
                    continue
                # print(f"===={bscp1} and unknown spectrum alignment {temp_res_index} have matching fragments, {len(fragment_res_list)}")

                # Filter b2 based on fragment results
                b2_df = b2_df.loc[fragment_res_list, :]
                b2_df["new_mass"] = np.around(np.array(fragment_res_list) / 1e4).astype(int)
                b2_df = b2_df.set_index("new_mass")
                b2_df = b2_df.groupby(b2_df.index).max()
                b2_df = mzbatch.to_relative_abundance(b2_df)
                b2_df = b2_df.mask(b2_df < 0.02, 0)
                b2_df = b2_df[(b2_df != 0).any(axis=1)]
                
                # Calculate similarity
                temp_b12_df = pd.concat([b1_df, b2_df], axis=1)
                temp_b12_df.columns = [-1, 0]
                temp_b12_df = temp_b12_df.fillna(0)
                temp_b12_df = mzbatch.to_relative_abundance(temp_b12_df)  #240805 convert to relative abundance!!, very important
                temp_b12_df = temp_b12_df.fillna(0)
                if sim_method == "cosine":
                    pos_sim_list = np.square(mzbatch.cal_pos_sim(temp_b12_df, sim_method=sim_method)).tolist()
                else:
                    pos_sim_list = mzbatch.cal_pos_sim(temp_b12_df, sim_method=sim_method)
                if Area13_2list == -1:
                    hengxiang204060_df.loc[n] = [sepuzhu, pos_neg, bscp1, b1_name, b1_adduct, ms2_index, ms2_mz, ms2_rt, pos_sim_list[0]] + [Area13_2list] * len(Area_cols_list)
                else:
                    hengxiang204060_df.loc[n] = [sepuzhu, pos_neg, bscp1, b1_name, b1_adduct, ms2_index, ms2_mz, ms2_rt, pos_sim_list[0]] + Area13_2list
                n += 1
    hengxiang204060_df.to_csv(f"../../Fulaoshi/230821_mzML_res/{analysis_path}/hengxiang204060_{sim_method}_df.csv")

In [ ]:
# Remove duplicates
for sepuzhu, pos_neg, sim_method in combine3_list[0:]:
    analysis_path = f"{analysis_batch}_{sepuzhu}_{pos_neg}"
    hengxiang204060_df = pd.read_csv(f"../../Fulaoshi/230821_mzML_res/{analysis_path}/hengxiang204060_{sim_method}_df.csv", index_col=0)
    hengxiang204060_nochong_df = hengxiang204060_df.drop_duplicates(subset=hengxiang204060_df.columns.tolist())
    hengxiang204060_nochong_df.to_csv(f"../../Fulaoshi/230821_mzML_res/{analysis_path}/deduplicated_hengxiang204060_{sim_method}_df.csv")


# Statistical unique term

In [48]:
def is_intersect(interval1, interval2):
    a, b = interval1
    c, d = interval2
    
    if max(a, c) <= min(b, d):
        return 1
    else:
        return 0

In [54]:
"""Reference file for loading statistics"""
static_ref_all_df = pd.read_excel("../../Fulaoshi/230821_mzML/ref_all-revise20240920.xlsx",index_col=0,sheet_name="Sheet2")
static_ref_all_df['batch_sampleid'] = static_ref_all_df['batch'].astype(str)+'/'+static_ref_all_df['sampleid'].astype(str)
static_ref_all_df['bscp']=static_ref_all_df['batch']+'+'+static_ref_all_df['sampleid'].astype(str)+'+'+static_ref_all_df['CAS No.'].astype(str)+'+'+static_ref_all_df['pos_neg']
static_ref_all_df.head(2)

,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,M.Wt,pos_neg,refer_RT,m/z,RT time,NL,bscp,Adduct,batch_sampleid,ev_nums,ev_list
1104,873_C18,C18,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,pos,1,282.1196,285.0,133000000,873_C18+26386+1867-73-8+pos,[M+H]+,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."
1106,873_C18,C18,26386,Corticosterone,50-22-6,C21H30O4,346.46,pos,1,347.2216,874.8,111000000,873_C18+26386+50-22-6+pos,[M+H]+,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."


In [ ]:
for sepuzhu, pos_neg, sim_method in combine3_list[0:]:
    Area_cols_list = Area_cols_dict[f"{sepuzhu}_{pos_neg}"]
    analysis_path = f"{analysis_batch}_{sepuzhu}_{pos_neg}"
    hengxiang204060_nochong_df = pd.read_csv(f"../../Fulaoshi/230821_mzML_res/{analysis_path}/deduplicated_hengxiang204060_{sim_method}_df.csv", index_col=0)
    hengxiang204060_nochong_df = hengxiang204060_nochong_df[hengxiang204060_nochong_df[Area_cols_list[0]] != -1]
    hengxiang204060_nochong_df.reset_index(inplace=True, drop=True)
    
    temp_hengxiang204060_df = hengxiang204060_nochong_df[Area_cols_list]
    temp_hengxiang204060_df = temp_hengxiang204060_df.astype(str)
    temp_hengxiang204060_df['Area_sum'] = temp_hengxiang204060_df.apply(lambda row: ''.join(row), axis=1)
    hengxiang204060_nochong_df["Area_sum"] = temp_hengxiang204060_df['Area_sum']
    
    res_groupby_df = res_df_dict[f"{sepuzhu}+{pos_neg}"]

    b1_ref_rt_list, b2_rt_left_list, b2_rt_right_list = [], [], []
    for index204060 in hengxiang204060_nochong_df.index[0:]:
        static_bscp1, static_b2_index = hengxiang204060_nochong_df.loc[index204060, ["bscp1", "b2_index"]]

        temp_ref_all_df = static_ref_all_df[static_ref_all_df['bscp'] == static_bscp1]
        if temp_ref_all_df.shape[0] == 0:
            b1_ref_rt_list.append(-100)  # Modified on 240920
        else:
            b1_ref_rt = temp_ref_all_df.loc[temp_ref_all_df.index[0], "RT time"]
            b1_ref_rt_list.append(b1_ref_rt)

        temp_res_groupby_df = res_groupby_df[res_groupby_df["ms2_index"] == static_b2_index]
        temp_b2_rt_df = temp_res_groupby_df.groupby(["ms2_index"]).agg(ref_rt_left=('ref_rt_left', "min"), ref_rt_right=('ref_rt_right', 'max')).reset_index()
        b2_rt_left, b2_rt_right = temp_b2_rt_df.loc[temp_b2_rt_df.index[0], ["ref_rt_left", "ref_rt_right"]]
        b2_rt_left_list.append(b2_rt_left)
        b2_rt_right_list.append(b2_rt_right)

    hengxiang204060_nochong_df["b1_ref_rt"] = b1_ref_rt_list
    hengxiang204060_nochong_df["b1_rt_left"] = hengxiang204060_nochong_df["b1_ref_rt"] - 15
    hengxiang204060_nochong_df["b1_rt_right"] = hengxiang204060_nochong_df["b1_ref_rt"] + 15
    hengxiang204060_nochong_df["b2_rt_left"] = b2_rt_left_list
    hengxiang204060_nochong_df["b2_rt_right"] = b2_rt_right_list
    hengxiang204060_nochong_df["has_intersect"] = hengxiang204060_nochong_df.apply(lambda x: is_intersect([x["b1_rt_left"], x["b1_rt_right"]], [x["b2_rt_left"], x["b2_rt_right"]]), axis=1)

    hengxiang_colist = list(hengxiang204060_nochong_df.columns)
    Area_first_col = "Area_sum"
    hengxiang204060_nochong_df["is_max_sim"] = 0
    max_similarity_indexes = hengxiang204060_nochong_df.groupby(by=[Area_first_col])['pos_sim'].idxmax()
    hengxiang204060_nochong_df.loc[max_similarity_indexes, "is_max_sim"] = 1
    hengxiang204060_nochong_df = hengxiang204060_nochong_df[hengxiang204060_nochong_df["b1_ref_rt"] != -100]
    hengxiang204060_nochong_df.to_csv(f"../../Fulaoshi/230821_mzML_res/{analysis_path}/static_deduplicated_hengxiang204060_{sim_method}_df.csv")


In [104]:
zong_static1_df = zong_df.groupby(groupby_list).agg(b2_index=('b2_index', list_agg), b2_mz=('b2_mz', list_agg),  
                                                    b2_rt_left=('b2_rt_left', 'min'),b2_rt_right=('b2_rt_right', 'max'),).reset_index()

,ref_ev,ref_mz,ref_rt_left,ref_rt_right,Area,ms2_ev,ms2_index,ms2_mz,ms2_rt,ms1_intensity,Area13_2list
13579,35,297.24216,45.18,83.34,104877925.0,204060,836,297.242218,55.972308,21361174.0,"[[59815727, 23964964, 12438672, 256215103, 102..."
13589,35,297.24216,45.54,80.40,104479167.0,204060,836,297.242218,55.972308,21361174.0,"[[59815727, 23964964, 12438672, 256215103, 102..."
13569,35,297.24219,46.26,81.24,107659395.0,204060,836,297.242218,55.972308,21361174.0,"[[59815727, 23964964, 12438672, 256215103, 102..."
13507,35,297.24222,44.88,83.64,104182764.0,204060,836,297.242218,55.972308,21361174.0,"[[59815727, 23964964, 12438672, 256215103, 102..."
13539,35,297.24222,46.02,83.28,106314221.0,204060,836,297.242218,55.972308,21361174.0,"[[59815727, 23964964, 12438672, 256215103, 102..."
13598,35,297.24222,46.26,83.52,105676604.0,204060,836,297.242218,55.972308,21361174.0,"[[59815727, 23964964, 12438672, 256215103, 102..."
13487,35,297.24225,45.24,81.36,106543262.0,204060,836,297.242218,55.972308,21361174.0,"[[59815727, 23964964, 12438672, 256215103, 102..."
13608,35,297.24225,46.20,83.04,106350927.0,204060,836,297.242218,55.972308,21361174.0,"[[59815727, 23964964, 12438672, 256215103, 102..."
13519,35,297.24228,40.20,82.32,109812085.0,204060,836,297.242218,55.972308,21361174.0,"[[59815727, 23964964, 12438672, 256215103, 102..."
13529,35,297.24228,45.12,83.22,106729430.0,204060,836,297.242218,55.972308,21361174.0,"[[59815727, 23964964, 12438672, 256215103, 102..."
